In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [212]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [172]:
#normalize data
tX, mean_x, std_x = standardize(tX)

In [213]:
tX.shape

(250000, 30)

In [174]:
#polynomial basis
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    degrees = np.arange(0, degree+1)
    psi = np.power(x[:, :, np.newaxis], degrees)
    return psi


In [177]:
xpoly = build_poly(tX,2)
xpoly.shape

(250000, 30, 3)

In [12]:
from helpers import *

In [13]:
def compute_loss(y, tx, w):
    e = y - np.dot(tx,w)
    N = len(y)
    L=e.T.dot(e)/(2*N)
    return L

In [69]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e=y-tx.dot(w)
    N=len(y)
    return -(np.transpose(tx).dot(e))/N

In [219]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    w = initial_w
    loss = np.inf
    for n_iter in range(max_iters):
        g = compute_gradient(y, tx, w)
        new_w = w - gamma*g;
        new_loss = compute_loss(y, tx, new_w)
        # print TO DELETE IN FINAL VERSION
        if new_loss <= loss:
            loss, w = new_loss, new_w
            gamma *=1.8 #accelerate algorithm learning rate
            #print("Gradient Descent({bi}/{ti}): loss={l} ; gamma={g}; gamma aumenta".format(
             # bi=n_iter, ti=max_iters - 1, l=loss, g= gamma))
        else:
            gamma *=0.3 #decelerate to avoid exponential growing
            #print("Gradient Descent({bi}/{ti}): loss={l} ; gamma={g}; gamma diminuisce".format(
             # bi=n_iter, ti=max_iters - 1, l=loss, g= gamma)
        print("Gradient Descent({bi}/{ti}): ||gradient||={grad}, loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, grad=np.linalg.norm(g), l=loss, w0=w[0], w1=w[1]))

    return w, loss

In [230]:
# Define the parameters of the algorithm.
max_iters = 200
gamma = 0.0001

# Initialization
w_initial = np.zeros(tX.shape[1])

# Start gradient descent.
gradient_w, gradient_loss = gradient_descent(y, tX, w_initial, max_iters, gamma)

Gradient Descent(0/199): ||gradient||=841.1056518611562, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(1/199): ||gradient||=478994.9203081395, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(2/199): ||gradient||=478994.9203081395, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(3/199): ||gradient||=478994.9203081395, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(4/199): ||gradient||=478994.9203081395, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(5/199): ||gradient||=478994.9203081395, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(6/199): ||gradient||=478994.9203081395, loss=20011.433828797766, w0=0.010766649510400012, w1=-0.0027284198603999992
Gradient Descent(7/199): ||gradient||=478994.9203081395, loss=1232.20

In [94]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    e=y-tx.dot(w)
    N=len(y)
    return -(np.transpose(tx).dot(e))/N

def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    w = initial_w
    loss = np.inf
    for n_iter in range(max_iters):
        for yn, xn in batch_iter(y, tx, batch_size):
            g = compute_stoch_gradient(yn, xn, w)
            new_w = w - gamma*g;
            new_loss = compute_loss(y, tx, new_w)
        if new_loss <= loss:
            loss , w = new_loss , new_w
            gamma *=1.8
            #print("SGD({bi}/{ti}): loss={l} ; gamma={g}; gamma aumenta".format(
             # bi=n_iter, ti=max_iters - 1, l=loss, g= gamma))
        else:
            gamma *=0.3
            #print("SGD({bi}/{ti}): loss={l} ; gamma={g}; gamma diminuisce".format(
             #     bi=n_iter, ti=max_iters - 1, l=loss, g= gamma))
        print("SGD({bi}/{ti}): |gradient|={grad}, loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, grad=np.linalg.norm(g), l=loss, w0=w[0], w1=w[1]))

    return w, loss

In [99]:
# from stochastic_gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.01
batch_size = 1

# Initialization
w_initial = np.zeros(tX.shape[1])

# Start SGD.
sgd_loss, sgd_w = stochastic_gradient_descent(
    y, tX, w_initial, batch_size, max_iters, gamma)


SGD(0/99): |gradient|=5.606944888677391, loss=0.6114707782168463, w0=0.006394104193840289, w1=0.005450759883688027
SGD(1/99): |gradient|=7.570340290732537, loss=0.4362372886140286, w0=0.0507102059786907, w1=-0.008130332381347041
SGD(2/99): |gradient|=3.573906656201934, loss=0.4362372886140286, w0=0.0507102059786907, w1=-0.008130332381347041
SGD(3/99): |gradient|=3.3552360099930136, loss=0.4362372886140286, w0=0.0507102059786907, w1=-0.008130332381347041
SGD(4/99): |gradient|=4.283991744813342, loss=0.43571977288915875, w0=0.049524230918568043, w1=-0.009027933449343517
SGD(5/99): |gradient|=3.214881116998168, loss=0.43571977288915875, w0=0.049524230918568043, w1=-0.009027933449343517
SGD(6/99): |gradient|=3.9249474696806628, loss=0.4354151761726099, w0=0.04857157669861133, w1=-0.00976244417735909
SGD(7/99): |gradient|=3.203119350249999, loss=0.4354151761726099, w0=0.04857157669861133, w1=-0.00976244417735909
SGD(8/99): |gradient|=3.2516736215578637, loss=0.4354151761726099, w0=0.0485715

In [150]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    D = tx.shape[1]
    G = tx.T.dot(tx)
    if(np.linalg.matrix_rank(G)==D):
        w = np.linalg.inv(G).dot(tx.T).dot(y)
    else:
        w = np.linalg.lstsq(G,tx.T.dot(y), rcond=None) [0]
    return w

In [153]:
# w_star = least_squares(y,xpoly)
xpoly[:,:,0]
w0 = least_squares(y,xpoly[:,:,0])
w1 = least_squares(y,xpoly[:,:,1])
w2 = least_squares(y,xpoly[:,:,2])

(250000, 30, 3)

In [ ]:
w = [w0, w1, w2]
yp = np.zeros((250000,3))
yp[:,0] = xpoly[:,:,0].dot(w0)
yp[:,1] = xpoly[:,:,1].dot(w1)
yp[:,2] = xpoly[:,:,2].dot(w2)
compute_loss(yp, xpoly, w)

In [193]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    N = len(y)
    G = tx.T.dot(tx)
    i = np.linalg.inv(G + 2*N*lambda_*np.eye(G.shape[0]))
    w_star = i.dot(tx.T).dot(y)
    return w_star

In [214]:
lambda_ = 0.2
w = ridge_regression(y, tX, lambda_)
loss = compute_loss(y, tX, w)
print(w, loss)

[ 2.24104142e-04 -8.68280397e-03 -2.68859353e-03 -2.22470926e-03
 -5.48363092e-03  5.00183793e-04 -1.72285231e-02  6.05199540e-02
  1.57457370e-05  3.49522005e-03 -5.39186900e-02  6.84999623e-02
  2.19603319e-02  5.70895097e-03 -4.23536533e-04 -1.28871636e-03
  3.30152909e-03 -5.20781620e-04  9.12327144e-04  4.86026909e-03
  4.62605701e-04 -7.55206472e-04 -5.59287480e-02  1.20180623e-03
 -6.34079801e-04 -4.00651108e-04  5.73198500e-05  1.25042999e-03
 -8.49579237e-04 -5.51323579e-03] 0.3483563858251794


## Generate predictions and save ouput in csv format for submission:

In [231]:
weights = gradient_w
weights

array([ 4.58964628e-04, -1.29132009e-03, -4.71611097e-04,  4.35105239e-04,
       -9.51234626e-05,  8.76108257e-04, -1.17527898e-04, -7.17490384e-06,
       -2.25082512e-04, -3.08541973e-04, -2.94446989e-05,  2.20370090e-05,
       -9.60401499e-05,  4.88644294e-04, -4.09816793e-07, -9.94246210e-07,
       -3.66728587e-04, -4.37996069e-07,  9.71191920e-07, -1.02244569e-05,
        1.36577033e-06, -6.03655237e-04, -1.23027882e-05,  1.17095806e-04,
        3.21211363e-05,  3.21108916e-05, -3.45141914e-04, -9.77186517e-05,
       -9.84173092e-05, -4.30457476e-04])

In [227]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [232]:
OUTPUT_PATH = '../data/submission.csv'
tX_test_s, _, _ = standardize(tX_test)
y_pred = predict_labels(weights, tX_test_s)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [233]:
loss = compute_loss(y_pred, tX_test_s, w)
loss

0.505418514821007